# DI Pre Processing
This notebook converts a video into frames. 
<br>
<br>
![UofC logo](../assets/images/uofc_logo-black.jpg)

In [9]:
#import libraries
import os
from dotenv import load_dotenv

In [10]:
# reload for module caching
from importlib import reload
import src.dataset.breathing_dataset
reload(src.dataset.breathing_dataset)
import src.transforms.breathing_transforms
reload(src.transforms.breathing_transforms)
import src.deep_learning_models.three_d_cnn
reload(src.deep_learning_models.three_d_cnn)

from src.dataset.breathing_dataset import BreathingDataset
from src.dataset.breathing_dataset_config import BreathingDatasetConfig
from src.transforms.breathing_transforms import train_transform
from src.deep_learning_models.three_d_cnn import Basic3DCNN

In [11]:
curr_dir = os.getcwd()

In [12]:
# load root directory
dotenv_path = os.path.join(curr_dir, ".env")
load_dotenv(dotenv_path)
root_path = os.getenv("ROOT_FOLDER")
input(f"Is this the right directory - {root_path}?")

# load user drive
user_drive = os.getenv("EXPORT_DRIVE")
local_export_path = os.getenv("EXPORT_PATH_LOCAL")
remote_export_path = os.getenv("EXPORT_PATH_REMOTE")

In [13]:
local_export_path

'E:/rgb'

## Run functions

In [14]:
# load data
train_path = local_export_path + "/Train"
train_dataset_config = BreathingDatasetConfig(train_path, "Without Blankets", None, train_transform, 100)

In [15]:
train_dataset_without_blankets = BreathingDataset(train_dataset_config)

In [16]:
"""
Testing data loader
"""
import torch.optim as optim
import torch.nn as nn

model = Basic3DCNN()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Number of epochs
num_epochs = 5

window_size = 10  # Example window size
step_size = 5    # Example step size

for epoch in range(num_epochs):
    running_loss = 0.0
    # Replace train_loader with dataset.sliding_window_generator
    for i, (inputs, labels) in enumerate(train_dataset_without_blankets.sliding_window_generator(window_size, step_size)):
        optimizer.zero_grad()
        inputs = inputs.permute(0, 2, 1, 3, 4)
        
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print and log statistics
        running_loss += loss.item()

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 5